In [1]:
from keras.layers import Dense,Lambda,Input,Dot,Add,Subtract,GaussianDropout
from keras.utils import Sequence,plot_model
from keras.models import Model,load_model
from keras import backend as K
import numpy as np

C:\Users\garyyu\AppData\Local\Continuum\anaconda3\envs\deeplearningcpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Denoising Autoencoder with weak supervision

In [2]:
# custommed function
def noisefunction(x):
    x_noise=K.ones_like(x)
    return K.in_train_phase(x_noise,x,1)

def tripletlossfunction(inputs):
    r01=inputs[0]
    r02=inputs[1]
    loss=K.log(1+K.exp(r02-r01))
    return loss

def losspassfunction(y_true,y_pred):
    return y_pred


# Build model

In [ ]:
x=Input((20000,))
y=GaussianDropout(rate=0.8,name="noise")(x)
y=Dense(units=100,activation="sigmoid",use_bias=False)(y)
encoder=Model(x,y)
x=Input((100,)) 
y=Dense(units=20000,activation="sigmoid")(x)
decoder=Model(x,y)
x0=Input((100,))
x1=Input((100,))
x2=Input((100,))
r01=Dot(axes=1)([x0,x1])
r02=Dot(axes=1)([x0,x2])
out=Lambda(tripletlossfunction)([r01,r02])
tripletloss=Model(inputs=[x0,x1,x2],outputs=out)
x0=Input((20000,))
x1=Input((20000,))
x2=Input((20000,))
h0=encoder(x0)
h1=encoder(x1)
h2=encoder(x2)
y0=decoder(h0)
y1=decoder(h1)
y2=decoder(h2)
yt=tripletloss([h0,h1,h2])
Tri_AutoEncoder=Model(inputs=[x0,x1,x2],outputs=[y0,y1,y2,yt])
Tri_AutoEncoder.compile(optimizer="adam",
                        loss=['binary_crossentropy','binary_crossentropy','binary_crossentropy',losspassfunction]
                       )

# Data

In [4]:
x0=np.random.randint(0,2,(32*3,20000))
x1=np.random.randint(0,2,(32*3,20000))
x2=np.random.randint(0,2,(32*3,20000))
yt=np.random.randint(0,2,(32*3,1))
X=[x0,x1,x2]
Y=[x0,x1,x2,yt]

# Train model

In [ ]:
Tri_AutoEncoder.fit(x=X,y=Y,epochs=5,batch_size=32)